# Capstone Project

## Finger Detection and Counting

## Importing Prereqisites

In [7]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

### Global Variables

In [8]:
background = None

accum_weight = 0.5

roi_t = 20
roi_b = 300
roi_r = 300
roi_l = 600

## Finding Average Background Value

The function calculates the weighted sum of input image source and accumulator distance so that distance becomes a running average of a frame sequence:

In [9]:
def calc_accumulate_average(frame, accum_weight):
    '''
    Given a frame and a previous accumulated weight, computes the weighted average of the image passed in.
    '''
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        retur None 
        
    cv2.accumulateWeighted(frame, background, accum_weight)

## Segment the Hand Region in Frame

In [11]:
def segment(frame, th_min=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    ret, threshold = cv2.threshold(diff, th_min, 255, cv2.THRESH_BINARY)

    image, contours, hierarchy = cv2.findContours(threshold.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours, key = cv2.contourArea)
        return (threshold, hand_segment)

## Counting Fingers with a Convex Hull

We just calculated the external contour of the hand. Now using that segmented hand, let's see how to calculate fingers. Then we can count how many are up!

Example of ConvexHulls:

<img src="Images/hand_convex.png">

In [12]:
def count_fingers(threshold, hand_segment):
    
    
    convhull = cv2.convexHull(hand_segment)
    
    top    = tuple(convhull[convhull[:, :, 1].argmin()][0])
    bottom = tuple(convhull[convhull[:, :, 1].argmax()][0])
    left   = tuple(convhull[convhull[:, :, 0].argmin()][0])
    right  = tuple(convhull[convhull[:, :, 0].argmax()][0])

    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2

    
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    max_distance = distance.max()
    
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)

    circular_roi = np.zeros(threshold.shape[:2], dtype="uint8")
    
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    circular_roi = cv2.bitwise_and(threshold, threshold, mask=circular_roi)

    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    count = 0

    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)

        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        
        if  out_of_wrist and limit_points:
            count += 1

    return count

## Run Program

In [13]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    ret, frame = cam.read()

    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    roi = frame[roi_t:roi_b, roi_r:roi_l]

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    if num_frames < 60:
        calc_accumulate_average(gray, accum_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVERAGE", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:
        hand = segment(gray)
        
        if hand is not None:
            threshold, hand_segment = hand
            cv2.drawContours(frame_copy, [hand_segment + (roi_r, roi_t)], -1, (255, 0, 0),1)
            fingers = count_fingers(threshold, hand_segment)
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Thesholded", threshold)

    cv2.rectangle(frame_copy, (roi_l, roi_t), (roi_r, roi_b), (0,0,255), 5)
    num_frames += 1

    cv2.imshow("Finger Count", frame_copy)

    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()